In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width: 98% }<\style>")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline